In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0.1
)

prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a helpful assistant. Answer the questions using only the following context. If you don't know the answer just say you don't know, don't make it up
     
    Context: {context}
     
    Previous conversation: {history}"""),
    ("human", "{question}")
])

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

In [17]:
# 문서 로딩 및 임베딩, 캐싱
loader = UnstructuredFileLoader("./files/document.txt")


docs = loader.load_and_split(text_splitter=splitter)


cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()


In [18]:
# 버퍼 메모리 적용
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


In [19]:
# chain 구성
final_chain = prompt | llm


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def invoke_chain(question):
    docs = retriver.get_relevant_documents(question)
    context = format_docs(docs)

    result = final_chain.invoke({
        "question": question,
        "context": context,
        "history": load_memory(None)
    })
    
    # 대화 내용을 memory에 저장
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    
    return result



In [20]:
invoke_chain("Is Aaronson guilty?")

AIMessage(content='Yes, according to the information provided in the context, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [21]:
invoke_chain("What message did he write in the table?")

AIMessage(content='He wrote "2+2=5" in the dust on the table.')

In [22]:
invoke_chain("Who is Julia?")

AIMessage(content='Julia is a character in the context provided. She is someone who Winston loves and cares for deeply.')

In [23]:
invoke_chain("What is previous conversation?")

AIMessage(content='The previous conversation was about Aaronson\'s guilt, the message Winston wrote on the table ("2+2=5"), and identifying Julia as a character Winston loves and cares for deeply.')

In [24]:
invoke_chain("What was the last question asked?")

AIMessage(content='The last question asked was "What is previous conversation?"')

In [25]:
invoke_chain("Is Aaronson guilty?")

AIMessage(content='Yes, Aaronson, along with Jones and Rutherford, were guilty of the crimes they were charged with.')